In [1]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *
from CutFunctions import *
from SystematicsHelpers import *

import ROOT
import array
import pickle
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

import warnings
warnings.filterwarnings("ignore")

Welcome to JupyROOT 6.24/06


<h1>Configuration Stuff Here</h1>

In [2]:
m = 200

ifSave = True
savePath = "../plot_files/06April2024_m"+str(m)+"_v3_systematics/"

<h1> Read in PKL Dataframe </h1>

<h3>HNL</h3>

In [ ]:
hnl_path = "../pkl_files/v3_April2024/df_m"+str(m)+"_v3_hnl.pkl"

file = open(hnl_path, 'rb')
df_hnl = pickle.load(file)
file.close
del hnl_path

In [ ]:
df_hnl = df_hnl[["run", "subrun", "event", "slc_id", "scale_pot", "simU", "scaledU", "mod_t", 'slc_comp', 'slc_true_event_type']]

<h3>TPC Neutrino</h3>

In [ ]:
nu_path = "../pkl_files/v3_April2024/df_m"+str(m)+"_v3_nu.pkl"

file = open(nu_path, 'rb')
df_nu = pickle.load(file)
file.close
del nu_path

In [ ]:
df_nu = df_nu[["run", "subrun", "event", "slc_id", "scale_pot", "mod_t", 'slc_comp', 'slc_true_event_type']]

<h3>Intime Cosmics</h3>

In [ ]:
cos_path = "../pkl_files/v3_April2024/df_m"+str(m)+"_v3_cos.pkl"

file = open(cos_path, 'rb')
df_cos = pickle.load(file)
file.close
del cos_path

In [ ]:
df_cos = df_cos[["run", "subrun", "event", "slc_id", "scale_pot", "mod_t", 'slc_comp', 'slc_true_event_type']]

<h1> Load Systematics Dataframe</h1>

<h3> HNL </h3>

In [ ]:
dfw_hnl = pd.DataFrame()

print(len(df_hnl))

for i in range(0, 4):
#for i in range(0, 1):
    hnl_path = "../pkl_files/v3_April2024/hnl_m"+str(m)+"_flxw_{}.pkl".format(i)
    print(hnl_path)
    
    file = open(hnl_path, 'rb')   
    df = pickle.load(file)
    
    #keep certain branches only
    branches = ['run','subrun','event','slc_id', 'slc_comp', 'slc_flux_weight_total'] + flux_list
    df = df[branches]
    
 
    df = df.merge(df_hnl, how='inner', on=['run','subrun','event','slc_id','slc_comp'])
    print(len(df))
    
    dfw_hnl = pd.concat((dfw_hnl, df), ignore_index=True)
    file.close()
    
del df_hnl

print(len(dfw_hnl))

<h3> TPC Neutrino </h3>

In [ ]:
dfw_nu = pd.DataFrame()

print(len(df_nu))

for i in range(0, 51):
#for i in range(0, 2):
    nu_path = "../pkl_files/v3_April2024/rockbox_flxw_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    #print(df.columns)
    
    df = df.merge(df_nu, how='inner', on=['run','subrun','event','slc_id', 'slc_comp'])
    print(len(df))
    
    dfw_nu = pd.concat((dfw_nu, df), ignore_index=True)
    file.close()
    
#del df_nu

print(len(dfw_nu))

<h3> NCPi0 </h3>

In [ ]:
dfw_ncpi0 = pd.DataFrame()

print(len(df_nu))

for i in range(0, 8):
    nu_path = "../pkl_files/v3_April2024/ncpi0_flxw_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    #print(df.columns)
    
    df = df.merge(df_nu, how='inner', on=['run','subrun','event','slc_id', 'slc_comp'])
    print(len(df))
    
    dfw_ncpi0 = pd.concat((dfw_ncpi0, df), ignore_index=True)
    file.close()
    
del df_nu

print(len(dfw_nu))

In [ ]:
dfw_nu_combined = pd.concat([dfw_ncpi0, dfw_nu])
print(len(dfw_nu_combined))

del dfw_nu
del dfw_ncpi0

In [ ]:
dfw_nu = dfw_nu_combined

del dfw_nu_combined

<h1>SAVE THIS DATAFRAME</h1>

In [ ]:
dfw_hnl.to_pickle("../pkl_files/v3_April2024/df_hnl_m"+str(m)+"_v3_weight.pkl", protocol = 5)
dfw_nu.to_pickle("../pkl_files/v3_April2024/df_nu_m"+str(m)+"_v3_weight.pkl", protocol = 5)
df_cos.to_pickle("../pkl_files/v3_April2024/df_cos_m"+str(m)+"_v3_weight.pkl", protocol = 5)

###### 